In [49]:
import requests
import json

In [50]:
def get_payload(doi):
    return {
        "doi": doi
    }

In [51]:
dois = [
    '10.1038/s41541-024-00898-w',
    '10.1016/j.eclinm.2024.102590',
    '10.1038/s41467-023-44276-x',
    '10.1016/s0140-6736(23)02467-4',
    '10.1093/pubmed/fdad196',
    '10.1016/j.vaccine.2023.07.070',
    '10.1016/j.lanepe.2023.100638'
]

In [62]:
#response = requests.post("http://localhost:8000/find",json=get_payload("10.1038/s41541-024-00898-w"))
response = requests.post("http://localhost:8000/find",json=get_payload(dois[0]))

In [63]:
data = response.json()['data']
print (json.dumps(data,indent=6))

{
      "doi": "10.1038/s41541-024-00898-w",
      "datasets": {
            "EAVE II": {
                  "id": 244,
                  "title": "Seasonal Influenza Vaccine Effectiveness II"
            },
            "ESoCiS": null
      }
}


In [65]:
import hashlib
from pyvis.network import Network
import networkx as nx
G = nx.Graph()

for i in range(len(dois)):
    G.add_node(dois[i], group=1)
    for j in range(i + 1, len(dois)):  # Start from i+1 to avoid duplicates and self-loops
        G.add_edge(dois[i], dois[j], group=1)


for doi in dois:
    response = requests.post("http://localhost:8000/find",json=get_payload(doi))
    if response.status_code != 200:
        continue
    data = response.json()['data']
    if not data:
        continue
    
    for key,entry in data['datasets'].items():
        if not entry:
            node_id = hashlib.md5(key.encode()).hexdigest()
            G.add_node(node_id, title=key, label=key, font={'color':'red'}, group=3)
            G.add_edge(data['doi'],node_id, group=3, color='red')
        else:
            node_id = f'dataset_{entry['id']}'
            title = entry['title']
            G.add_node(node_id, title=title, label=title, group=3)
            G.add_edge(data['doi'],node_id, group=3)

len(G.nodes)

18

In [66]:
net = Network(notebook=True)
net.from_nx(G)

net.barnes_hut(
    gravity=-2000,
    central_gravity=0.3,
    spring_length=95,
    spring_strength=0.04,
    damping=0.09,
    overlap=0.1
)

net.show("./temp.html")

./temp.html
